# GISY6400 Capstone - Wildfire Hazard: Seasonal Rasters, Normalized Weighted Sum

Program: wildfire_totHazard.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook generates seasonal wildfire hazard rasters using fuel, terrain and weather inputs. 
Date: May 17, 2025

### Import Python Modules

In [ ]:
import arcpy
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")

### Normalize Rasters and Create Weighted Sum by Season

In [ ]:
def normalize_raster(raster):
    """Normalize raster values to range 1-5"""
    min_val = float(arcpy.GetRasterProperties_management(raster, "MINIMUM").getOutput(0))
    max_val = float(arcpy.GetRasterProperties_management(raster, "MAXIMUM").getOutput(0))
    if max_val == min_val:
        return raster
    normalized = ((raster - min_val) / (max_val - min_val)) * 4 + 1
    return normalized

def calculate_hazard(fuel_path, terrain_path, weather_path, output_path,
                     w_fuel=0.4, w_terrain=0.3, w_weather=0.3):
    # Load rasters
    fuel_raster = Raster(fuel_path)
    terrain_raster = Raster(terrain_path)
    weather_raster = Raster(weather_path)

    # Normalize fuel (0-4 scale, adjust to 1-5)
    fuel_norm = fuel_raster + 1

    # Terrain assumed already 1-5, no normalization needed
    terrain_norm = terrain_raster

    # Normalize weather raster to 1-5
    weather_norm = normalize_raster(weather_raster)

    # Calculate weighted sum hazard
    hazard = (fuel_norm * w_fuel) + (terrain_norm * w_terrain) + (weather_norm * w_weather)

    # Save output raster
    hazard.save(output_path)
    print(f"Hazard raster saved to: {output_path}")

# Base paths
base_gdb = r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb"
fuel_path = f"{base_gdb}\\fuel_final"
terrain_path = f"{base_gdb}\\terrain_final"

# Calculate spring hazard
calculate_hazard(
    fuel_path=fuel_path,
    terrain_path=terrain_path,
    weather_path=f"{base_gdb}\\FWHI_spring_mask",
    output_path=f"{base_gdb}\\hazard_spring"
)

# Calculate summer hazard
calculate_hazard(
    fuel_path=fuel_path,
    terrain_path=terrain_path,
    weather_path=f"{base_gdb}\\FWHI_summer_mask",
    output_path=f"{base_gdb}\\hazard_summer"
)

# Calculate fall hazard
calculate_hazard(
    fuel_path=fuel_path,
    terrain_path=terrain_path,
    weather_path=f"{base_gdb}\\FWHI_fall_mask",
    output_path=f"{base_gdb}\\hazard_fall"
)

### Reclassify Rasters with Equal Interval

In [ ]:
def reclassify_to_4_classes(input_raster_path, output_raster_path):
    remap = RemapRange([
        [1, 2, 1],
        [2, 3, 2],
        [3, 4, 3],
        [4, 5.01, 4]
    ])
    raster = Raster(input_raster_path)
    reclass_raster = Reclassify(raster, "Value", remap, "NODATA")
    reclass_raster.save(output_raster_path)
    print(f"Reclassified raster saved to: {output_raster_path}")

base_gdb = r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb"

for season in ["spring", "summer", "fall"]:
    in_raster = f"{base_gdb}\\hazard_{season}"
    out_raster = f"{base_gdb}\\hazard_{season}_reclass"
    reclassify_to_4_classes(in_raster, out_raster)

arcpy.CheckInExtension("Spatial")